In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [65]:
from malaya.text.function import (
    check_ratio_numbers,
    check_ratio_punct,
    is_emoji,
    is_laugh,
    is_mengeluh
)
import malaya

In [3]:
vocab, embedded = malaya.wordvector.load(model = 'socialmedia')
wordvector = malaya.wordvector.WordVector(embedded, vocab)

Load pretrained wordvector into `malaya.wordvector.WordVector` class will disable eager execution.
2023-06-03 13:30:39.937838: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 13:30:39.980921: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-03 13:30:39.980946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2023-06-03 13:30:39.980949: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2023-06-03 13:30:39.981015: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.182.3
2023-06-

In [14]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=31, metric='cosine').fit(
    wordvector._embed_matrix
)

In [16]:
words = ['jokowi', 'prabowo', 'soekarno', 'soeharto', 'wiranto', 'anjeng', 'goblog', 'kontol',
        'pakde', 'bisa', 'ngasilin', 'ngapain', 'gemer', 'partai', 'gue', 'uang', 'kalo',
        'pake', 'kampret', 'ngubah', 'antek', 'aseng','brengsek', 'pakbis', 'ngehe', 'kontol',
        'bodoamat', 'walikota', 'donk' ,'pamer', 'ngapa2in', 'tiduran',
        'ngomong', 'disholatin', 'bngt', 'bangkeknye', 'brengsek', 'pakbis', 'ngehe',
        'konyol', 'cewe', 'segede']

similars = []
for word in words:
    print(word)
    
    try:
        distances, idx = nn.kneighbors(
            wordvector.get_vector_by_name(word).reshape((1, -1))
        )
        word_list = []
        for i in range(1, idx.shape[1]):
            word_list.append(
                [wordvector._reverse_dictionary[idx[0, i]], 1 - distances[0, i]]
            )
        similars.append(word_list)
    except Exception as e:
        print(e)

jokowi
prabowo
soekarno
soeharto
wiranto
anjeng
goblog
kontol
pakde
bisa
ngasilin
ngapain
gemer
partai
gue
uang
kalo
pake
kampret
ngubah
antek
aseng
brengsek
pakbis
input not found in dictionary, here top-5 nearest words [pakis, pakbin, pakbim, pakboi, pakbos]
ngehe
kontol
bodoamat
walikota
donk
pamer
ngapa2in
tiduran
ngomong
disholatin
bngt
bangkeknye
input not found in dictionary, here top-5 nearest words [bangkenye, bangkek, bangkee, bangkenya, bangkekk]
brengsek
pakbis
input not found in dictionary, here top-5 nearest words [pakis, pakbin, pakbim, pakboi, pakbos]
ngehe
konyol
cewe
segede


In [94]:
# !wget https://raw.githubusercontent.com/ardwort/freq-dist-id/master/data/twitter.csv

In [95]:
from glob import glob

vocab_files = glob('indonlu/dataset/*/vocab_uncased.txt')

In [96]:
vocabs = []
for f in vocab_files:
    with open(f) as fopen:
        v = fopen.read().split('\n')
    vocabs.extend(v)
vocabs = list(filter(None, vocabs))

In [97]:
import enchant
d = enchant.Dict('en_US')

is_english = lambda x: d.check(x)

In [98]:
import pandas as pd

df = pd.read_csv('twitter.csv', header = None)
df.head()

,0,1,2
0,di,404300,1.162798
1,yg,279458,0.803743
2,ya,270836,0.778945
3,aku,250220,0.719652
4,yang,240116,0.690592


In [99]:
def reject(w):
    if malaya.dictionary.is_malay(w):
        return True
    if malaya.dictionary.is_english(w):
        return True
    if is_english(w):
        return True
    return False

def reject_vocab(w):
    try:
        if check_ratio_numbers(w) > 0.4:
            return True
        if check_ratio_punct(w) > 0.4:
            return True
        if is_emoji(w):
            return True
        if is_laugh(w):
            return True
        if is_mengeluh(w):
            return True
        if w.startswith('</') and w.endswith('>'):
            return True
        if reject(w):
            return True
        return False
    except:
        return True

In [100]:
df = set(df[0].dropna().tolist())
len(df)

9999

In [101]:
indon_words = [w for w in df if not reject_vocab(w)]
len(indon_words)

2855

In [102]:
vocab_indon = [w for w in vocabs if not reject_vocab(w)]
len(vocab_indon)

50632

In [103]:
_list_laughing = {
    'huhu',
    'haha',
    'gaga',
    'hihi',
    'wkawka',
    'wkwk',
    'kiki',
    'keke',
    'huehue',
    'hih',
    'waka',
    'ckck',
    'hehe',
    'lolol',
    'lol',
    'wkaka'
}

In [211]:
import itertools
import re

merged = list(itertools.chain(*similars))
indon_socialmedia_words = set([i[0] for i in merged if len(i[0]) > 2 and not len(re.findall(r'\d+',i[0]))])
indon_socialmedia_words = indon_socialmedia_words | set(words) | set(indon_words) | df
indon_socialmedia_words = {i for i in indon_socialmedia_words if type(i) == str}
indon_socialmedia_words = {i for i in indon_socialmedia_words if len(i) > 2}
indon_socialmedia_words = {
        word
        for word in indon_socialmedia_words
        if not any([laugh in word for laugh in _list_laughing])
        and word[: len(word) // 2] != word[len(word) // 2 :]
}

minus = list([w for w in indon_socialmedia_words if not reject_vocab(w) and len(w) > 3 and w[-1] != '2'])
len(minus)

2363

In [212]:
rejects = ['yup','babi', 'tanda','blajar', 'aah','iyaa','mana','smbil','knpa',
          'adekku', 'bigbang','masa','dropbox','scumbag',
          'wakaka', 'hmmm', 'emoticon','mana', 'bangun','ehm',
          'sia','ehh', 'mey', 'marah','isn','duh','eek',
          'akuu','twiter','twitter','ferrari','hahha',
          'mlayu','woyy','ost','yaa','lgs','otw','okeee',
          'waah','waaah','iyaa','eee','berenti','iphone',
          'mbayar','mbahas','brenti','tanya','yeee','ayooo','kaaak',
          'hooh','haha','hha','yaaa','faisal','pliss','rvp',
          'ciee','fifa','kkk','argh','tweetnya','oon','lagi',
          'loooh','looh','everytime','10rb','gaza',
          'entahlah','amin','gile','crita','besok',
          'aaakk','singap','lama','kemana','jangan',
          'usb','moh','eeh','gsm','nihh','amin',
          'haaa','akhir','nego','bljr','bts',
          'jeh','kih','xdd','guw','adele','uhuy',
          'apa','bgitu','amiin','amiiin','mantab',
          'bored','tmpt','skt','tau','laen','tidor',
          'tuhh','cari','sblm','baca','setaun','tmbh',
          'mkan','wassap','innali','htc','zzz','ahhh',
          'banyak2','mancing','main','jgn','lohh',
          'itu','jauh','ha3','hugs','anj','ntah',
          'aww','kfc','pkai','kawan','tdi',
          'dulu','kali2','oww','acc','waa','dtg',
           'bgn','fyi','inii','yey','yah',
          'lgi','syg','eaa','bnyk','ngaku2',
          'wtf','bukak','hii','pfft','obama','aaak',
          'mmm','hbo','yeay','huaa','tlg','malu','avenger',
          'knp','php','oppa','broo','hus','sucks','weh','rmh','bln','cmn','wib','ihh',
          'ics','hardisk','ooh','yooo','beres','yamaha','seo','smua','mauu','pagi',
          'satu2','heu','hhe','skrng','err', 'ronaldo','cita','omg','socmed',
          'ktm','uitm','nyet','ahh','dkt','ikut','klau','nikmat','guling','gpp',
          'ubuntu','krja','fav','bkn','makan','uhuk','pesbuk','slalu','retweets',
          'xperia','smoga','messi','friendzone','soundcloud','bagi', 'bro',
          'spiderman', 'unfollow', 'mozilla', 'beatles', 'sama2',
          'agak2', 'sayangku', 'ɔˇ³ˇ','º̩̩̀', 'untungnya', 'ketawanya', 'spain',
          'istri', '̩̩̩͡', 'sony', 'hadiahnya', 'laki2', 'coldplay', 'italy',
          'linux', 'watsap','woiii','orang2', 'jalan2', 'hati2', 'farah', 'dslr', 'dihatimu',
          'linkedin', 'rata2', 'dadah2', '̮ơ̴̴͡', 'ɨɨɑɨɨɑɨɨɑ', 'nobita', 'disini', 'kaan',
          'sekalii', 'endingnya', 'tapinya']

minus = [m for m in minus if all([r not in m for r in rejects])]

In [213]:
minus.extend(['takbisa', 'ngapen', 'paskabayar', 'kok', 'sampe', 'asin', 'gurih', 'sih',
             'buk', 'neng', 'uang', 'pacar', 'jasmu', 'ujung', 'telen', 'banget',
             'bisa', 'pengen', 'gue', 'wong', 'ganteng', 'gwe', 'apaan', 'dengerinnya',
             'dengerinya', 'dengerin', 'fonsel'])

In [214]:
len(minus)

2016

In [215]:
malaya.dictionary.is_malay('dijalan')

False

In [216]:
minus

['dijalan',
 'mwalfin',
 'karokean',
 'colokan',
 'kode',
 'iyoo',
 'weleh',
 'sante',
 'moso',
 'emaknya',
 'kosannya',
 'temenku',
 'ndusel',
 'namamu',
 'iseh',
 'jogjakarta',
 'dicek',
 'palestina',
 'ngok',
 'nabrak',
 'ngebut',
 'kalibata',
 'ndue',
 'patrakars',
 'hajar',
 'durung',
 'kantong',
 'enggres',
 'utang',
 'beib',
 'dibilangin',
 'ngepet',
 'ngarti',
 'wedok',
 'dimaksud',
 'bagusan',
 'jelasin',
 'ndah',
 'sitik',
 'seriusan',
 'ujannya',
 'anying',
 'couldn',
 'onlen',
 'yaiyalah',
 'mboten',
 'keur',
 'pfffttt',
 'glek',
 'santet',
 'mentri',
 'mbak',
 'godain',
 'kapolri',
 'kelaperan',
 'keringet',
 'mboh',
 'patjar',
 'fatmawati',
 'nggilani',
 'rene',
 'bebeb',
 'ndang',
 'jenenge',
 'solusinya',
 'denpasar',
 'ndra',
 'aceng',
 'krik',
 'ksana',
 'yutub',
 'ngana',
 'ospek',
 'handphone',
 'nomor',
 'pemda',
 'rezim',
 'wisuda',
 'minal',
 'hayu',
 'kesini',
 'percoyo',
 'maksa',
 'teroris',
 'keselek',
 'ngendus',
 'maning',
 'pikirin',
 'salafi',
 'wenger',


In [218]:
import json

with open('indon-words-socialmedia.json', 'w') as fopen:
    json.dump(minus, fopen)